Hopefully, This script will allow me to load up and fix up the moored data

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import netCDF4 as nc
import datetime as dt
from salishsea_tools import evaltools as et, viz_tools
import gsw 
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.dates as mdates
import cmocean as cmo
import scipy.interpolate as sinterp
import pickle
import cmocean
import json
import f90nml
from collections import OrderedDict
from matplotlib.colors import LogNorm

fs=16
mpl.rc('xtick', labelsize=fs)
mpl.rc('ytick', labelsize=fs)
mpl.rc('legend', fontsize=fs)
mpl.rc('axes', titlesize=fs)
mpl.rc('axes', labelsize=fs)
mpl.rc('figure', titlesize=fs)
mpl.rc('font', size=fs)
mpl.rc('font', family='sans-serif', weight='normal', style='normal')

import warnings
#warnings.filterwarnings('ignore')
from IPython.display import Markdown, display

%matplotlib inline

In [4]:
saveloc='/ocean/kflanaga/MEOPAR/phytoData'
year=2018

In [5]:
df = pd.read_csv(f'{saveloc}/KingCounty_Marine_Phytoplankton_Samples_by_Taxonomic_Group.csv')

In [12]:
df.columns

Index(['Locator', 'Site Name', 'Lon', 'Lat', 'Sample Number', 'Collect Date',
       'Depth (m)', 'Taxonomic Group', 'Taxon', 'Rank', 'Kingdom', 'Phylum',
       'Class', 'Order', 'Family', 'Genus', 'Species', 'Biovolume (mm^3/L)',
       'Abundance (particles/mL)', 'Quality', 'Steward Note', 'Sample ID',
       'Grab ID', 'Profile ID', 'Replicates', 'Replicates Of', 'DataSource'],
      dtype='object')

In [13]:
df['Taxonomic Group'].unique()

array(['Centric diatoms', 'dinoflagellates', 'Amylax triacantha',
       'Pennate diatoms', 'Ciliates', 'Dinoflagellates',
       'Silicoflagellates', 'Centric diatom', 'Flagellates', 'Diatoms',
       'Brown algae', 'Haptophytes', 'Unidentified ',
       'Unidentified dinoflagellates', 'Unidentified zooplankton'],
      dtype=object)

In [15]:
df

,Locator,Site Name,Lon,Lat,Sample Number,Collect Date,Depth (m),Taxonomic Group,Taxon,Rank,...,Biovolume (mm^3/L),Abundance (particles/mL),Quality,Steward Note,Sample ID,Grab ID,Profile ID,Replicates,Replicates Of,DataSource
0,JSUR01,Point Wells,NaN,NaN,L68492-2,09/05/2017,1.10,Centric diatoms,Actinoptychus senarius,Species,...,0.10750,0.73730,1,NaN,36197,36197.0,29122,NaN,NaN,KCEL
1,JSUR01,Point Wells,NaN,NaN,L69494-2,03/07/2018,1.00,Centric diatoms,Actinoptychus senarius,Species,...,0.18620,0.62350,1,NaN,154747,154747.0,67895,NaN,NaN,KCEL
2,JSUR01,Point Wells,NaN,NaN,L72509-2,06/17/2019,1.00,Centric diatoms,Actinoptychus senarius,Species,...,0.32350,1.01100,1,NaN,172548,172548.0,75611,NaN,NaN,KCEL
3,JSUR01,Point Wells,NaN,NaN,L62667-3,05/18/2015,0.99,Centric diatoms,Actinoptychus senarius,Species,...,0.41340,0.09531,2,NaN,528,528.0,14,NaN,NaN,KCEL
4,JSUR01,Point Wells,NaN,NaN,L72418-2,06/03/2019,1.10,Centric diatoms,Actinoptychus senarius,Species,...,1.18800,1.13200,1,NaN,172539,172539.0,75608,NaN,NaN,KCEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23630,NSEX01,East Passage,NaN,NaN,L75380-9,09/09/2020,0.99,Unidentified zooplankton,unidentified zooplankton,NaN,...,0.96970,1.22500,1,NaN,193159,193159.0,80651,NaN,NaN,KCEL
23631,NSEX01,East Passage,NaN,NaN,L75498-9 DUP,09/22/2020,1.00,Unidentified zooplankton,unidentified zooplankton,NaN,...,0.25760,0.11850,1,NaN,193906,193719.0,80813,NaN,NaN,KCEL
23632,NSEX01,East Passage,NaN,NaN,L75498-9,09/22/2020,1.00,Unidentified zooplankton,unidentified zooplankton,NaN,...,0.13910,0.15510,1,NaN,193893,193719.0,80813,NaN,NaN,KCEL
23633,NSEX01,East Passage,NaN,NaN,L75813-9,11/03/2020,1.00,Unidentified zooplankton,unidentified zooplankton,NaN,...,0.08850,0.02792,1,NaN,193922,193922.0,80982,NaN,NaN,KCEL


I think that the conversion will from northing to easting will not be too hard. The example Susan gave is pretty good. 

''' gislats, gislons = transformer.transform(xx=xx, yy=yy)
plt.scatter(gislons, gislats) '''

is particulalry useful looking. 

In [40]:
##### Saving data as Pickle files to be used in the summary file
saveloc='/ocean/kflanaga/MEOPAR/mooredData'
with open(os.path.join(saveloc,f'data_{Mooring}_{year}.pkl'),'wb') as hh:
    pickle.dump(df,hh)
    
with open(os.path.join(saveloc,f'daily_data_{Mooring}_{year}.pkl'),'wb') as hh:
    pickle.dump(df_daily_avg,hh)